In [1]:
import pandas as pd
import json
import ijson
#pip install fsspec
#pip install openpyxl

### 1. Loading Jason Data to Phyton data frame 

In [2]:
#Load Jason data to dataframe, 
#to read from the file at a time to manage memory use

# Define the path to each JSON file
file_paths = [
    'ais-processed-log-2019-12.json', # Dec19 4.7 million rows
    'ais-processed-data_location_202001_filtered.json', #prefiltered Jan20 3.61 million rows as an alternative if original Jan20 that can not be processed
    'ais-processed-log-2020-12.json', # Dec20 3.4 million rows
    'ais-processed-log-2021-01.json', # Jan21 2.6 million rows
#    'ais-processed-log-2020-01.json' # Jan20 12 million rows
    ]

# Initialize an empty list to store data
data_list = []
error_count = 0

# Chunk size for processing data
chunk_size = 1000000

# Iterate through each file and load the data
for file_path in file_paths:
    with open(file_path, 'r') as file:
        try:
            data = json.load(file)
            if isinstance(data, list):
                data_list.extend(data)
            elif isinstance(data, dict):
                data_list.append(data)
        except json.JSONDecodeError as e:
            error_count += 1

    # Convert the list of dictionaries to a DataFrame
    df_123_123 = pd.DataFrame(data_list)

    # Print error count and the number of rows for each file
    print(f"File: {file_path}, Cummulative Error Count: {error_count}, Cummulative Rows: {df_123.shape[0]}")

#print('error:', error_count)
#print('df.shape', df.shape)


File: ais-processed-log-2019-12.json, Cummulative Error Count: 0, Cummulative Rows: 4700810
File: ais-processed-data_location_202001_filtered.json, Cummulative Error Count: 0, Cummulative Rows: 8314545
File: ais-processed-log-2020-12.json, Cummulative Error Count: 0, Cummulative Rows: 11762425
File: ais-processed-log-2021-01.json, Cummulative Error Count: 0, Cummulative Rows: 14380370


In [6]:
#Descriptive Statistics of Original Dataframe
df.describe().round(0).T

,count,mean,std,min,25%,50%,75%,max
type,14380370.0,2.0,4.0,1.0,1.0,1.0,1.0,19.0
mmsi,14380370.0,523021033.0,97682163.0,0.0,538003713.0,563051400.0,565209000.0,999486810.0
speed,13831579.0,3.0,5.0,0.0,0.0,0.0,6.0,102.0
accuracy,13946389.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
lon,13946389.0,104.0,7.0,-180.0,104.0,104.0,104.0,181.0
lat,13946389.0,2.0,6.0,-90.0,1.0,1.0,1.0,91.0
course,13032313.0,182.0,108.0,0.0,84.0,193.0,281.0,360.0
heading,9519653.0,164.0,106.0,0.0,71.0,128.0,270.0,359.0
status,13110203.0,2.0,4.0,0.0,0.0,0.0,3.0,15.0
turn,13110203.0,-37.0,69.0,-128.0,-128.0,0.0,0.0,127.0


### 2. Cleaning The Data

####    2.1. Filter only message type of 1, 2, 3 since we predetermine it as our scope

In [11]:
# Create dataframe df with only type 1, 2, 3 inside 
df_123 = df[df['type'].isin([1,2,3])]

# Comparison before and after (for checking)
b = df.shape
a = df_123.shape
diff_rows = a[0] - b[0]
diff_cols = a[1] - b[1]
print('Data type 1,2, and 3')
print('before: ',b)
print('after : ', a)
print('row differences: ', diff_rows)
print('columns differences: ', diff_cols)
print()

Data type 1,2, and 3
before:  (14380370, 25)
after :  (13110203, 25)
row differences:  -1270167
columns differences:  0



In [16]:
#Descriptive Statistics

df_123.describe().round(0).T

,count,mean,std,min,25%,50%,75%,max
type,13110203.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0
mmsi,13110203.0,523457541.0,97628566.0,0.0,538003713.0,563066530.0,565282000.0,999486810.0
speed,12995930.0,3.0,5.0,0.0,0.0,0.0,6.0,102.0
accuracy,13110203.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
lon,13110203.0,104.0,7.0,-180.0,104.0,104.0,104.0,181.0
lat,13110203.0,2.0,6.0,-90.0,1.0,1.0,1.0,91.0
course,12276407.0,187.0,106.0,0.0,92.0,199.0,283.0,360.0
heading,9505001.0,164.0,106.0,0.0,71.0,128.0,271.0,359.0
status,13110203.0,2.0,4.0,0.0,0.0,0.0,3.0,15.0
turn,13110203.0,-37.0,69.0,-128.0,-128.0,0.0,0.0,127.0


#### 2.2. Remove all columns which have all nol values

In [15]:
# Drop columns with all null values
df_123= df_123.drop(df_123.columns[df_123.isnull().all()], axis=1)

#### 2.3. Remove all zero rows in given columns

In [104]:
# Drop rows with Null values in given columns 

# Drop rows with missing values
#df_123 = df_123.dropna(subset=['speed', 'course', 'heading'])
df_123 = df_123.dropna(subset=['speed', 'course'])
a = df_123.shape

# Comparison before and after (For checking)
diff_rows = a[0] - b[0]
diff_cols = a[1] - b[1]
print('df_123')
print('before: ', b)
print('after : ', a)
print('row differences: ', diff_rows)
print('columns differences: ', diff_cols)

df_123
before:  (14380370, 25)
after :  (9239006, 25)
row differences:  -5141364
columns differences:  0


#### 2.4. Convert column 'time' to date format and break it to time and date

In [ ]:
# Convert 'time' column to datetime format
df_123['time'] = pd.to_datetime(df_123['time'])

# Create column 'time_'  and 'date_' column make a specific datetime format
df_123['time_'] = df_123['time'].dt.strftime('%H:%M')
df_123['date_'] = df_123['time'].dt.strftime('%y/%m/%d')

df_123[['time','time_','date_']]

,time,time_,date_
0,2019-12-01 00:00:30+00:00,00:00,2019-12-01
1,2019-12-01 00:08:36+00:00,00:08,2019-12-01
2,2019-12-01 00:09:01+00:00,00:09,2019-12-01
3,2019-12-01 00:19:26+00:00,00:19,2019-12-01
4,2019-12-01 00:19:49+00:00,00:19,2019-12-01
...,...,...,...
6653543,2021-01-31 23:13:15+00:00,23:13,2021-01-31
6653544,2021-01-31 23:15:35+00:00,23:15,2021-01-31
6653545,2021-01-31 23:18:45+00:00,23:18,2021-01-31
6653546,2021-01-31 23:27:13+00:00,23:27,2021-01-31


#### 2.5. Remove missing values in columns of interest

In [33]:
# Replacing null values with specified values
df_123['heading'].fillna(511, inplace=True)
df_123['speed'].fillna(102.3, inplace=True)
df_123['course'].fillna(360.0, inplace=True)

df_123.describe().round(0)

,type,mmsi,speed,accuracy,lon,lat,course,heading,status,turn,shipId
count,6653548.0,6653548.0,6653548.0,6653548.0,6653548.0,6653548.0,6653548.0,6653548.0,6653548.0,6653548.0,6653548.0
mean,1.0,509572259.0,4.0,0.0,104.0,1.0,190.0,199.0,1.0,-13.0,4322.0
std,1.0,106446245.0,6.0,0.0,4.0,2.0,108.0,148.0,2.0,52.0,1664.0
min,1.0,205408000.0,0.0,0.0,-179.0,-90.0,0.0,0.0,0.0,-128.0,2.0
25%,1.0,477480100.0,0.0,0.0,104.0,1.0,92.0,75.0,0.0,0.0,3408.0
50%,1.0,563075100.0,0.0,0.0,104.0,1.0,201.0,141.0,0.0,0.0,5041.0
75%,1.0,565603000.0,7.0,1.0,104.0,1.0,285.0,299.0,0.0,0.0,5473.0
max,3.0,710239000.0,102.0,1.0,181.0,91.0,360.0,511.0,15.0,127.0,6791.0


#### 2.6. Remove abnormal GPS coordinate and add location's information based on 1 nautical grid  

In [ ]:
# Drop rows when lon and lat outside the given range 
df_123 = df_123[(df_123['lon'].between(103.38, 103.94)) & (df_123['lat'].between(0.83, 1.39))]

#Create column Geo_Cell_ID with looping function
# Create an example DataFrame
#df1 = pd.DataFrame({
#    'lon': [103.4, 103.7, 103.9, 103.5],
#    'lat': [1.1, 0.9, 1.3, 0.5]
#})

# Create the grid
lon_grid = [i * 0.01 + 103.38 for i in range(int((103.94 - 103.38) / 0.01))]
lat_grid = [i * 0.01 + 0.83 for i in range(int((1.39 - 0.83) / 0.01))]

# Create a function for the filter
def encoding_filter(lon, lat):
    if (103.38 <= lon < 103.94) and (0.83 <= lat < 1.39):
        lon_index = int((lon - 103.38) / 0.01)
        lat_index = int((lat - 0.83) / 0.01)
        return lat_index * len(lon_grid) + lon_index + 1
    else:
        return None

# Apply the filter to df_123
df_123['Geo_Cell_ID'] = df_123.apply(lambda row: encoding_filter(row['lon'], row['lat']), axis=1)

# Print the example
print(df_123[['lat','lon','Geo_Cell_ID']])


             lat        lon  Geo_Cell_ID
0        1.13256  103.75723       1718.0
1        1.15364  103.78775       1833.0
2        1.15550  103.79072       1834.0
3        1.17622  103.83163       1950.0
4        1.17681  103.83324       1950.0
...          ...        ...          ...
6653543  1.26823  103.77611       2448.0
6653544  1.26950  103.77640       2448.0
6653545  1.27108  103.77667       2504.0
6653546  1.27450  103.77713       2504.0
6653547  1.27623  103.77578       2504.0

[6631022 rows x 3 columns]


#### 27. Include ShipID in the data frame

In [24]:
#Load vessel dimension in to phuton that contains ship ID number
df_imo = pd.read_excel('unique_mmsi_imo_values_v2.xlsx')
df_imo

,shipId,imo,mmsi,vessel_name,callsign
0,1,9726671,105792957,MAERSK STADELHORN,9V5223
1,2,9235268,205408000,SA EUROPE,ONCP
2,3,9230050,205421000,EXCALIBUR,ONCE
3,4,9444649,205553000,EXEMPLAR,ONFZ
4,5,9416733,205559000,FRATERNITY,ONGB
...,...,...,...,...,...
6786,6787,9453872,710029870,MACHADO DE ASSIS,PPBC
6787,6788,9453884,710032130,MILTON SANTOS,PPBL
6788,6789,9547673,710033180,CASTRO ALVES,PU2179
6789,6790,9547685,710033240,CARLOS DRUMMOND DE ANDRADE,PU3965


In [28]:
# Adding ship ID by joining main data frame df and vessel information data frame 
df_123 = df_123.merge(df_imo[['mmsi', 'shipId']], on=['mmsi'], how='inner') #include only ships that have shipId

# Comparison before and after (For checking)
a = df_123.shape
diff_rows = a[0] - b[0]
diff_cols = a[1] - b[1]
print('df_123')
print('before: ', b)
print('after : ', a)
print('row differences: ', diff_rows)
print('columns differences: ', diff_cols)


df_123
before:  (14380370, 25)
after :  (6653548, 12)
row differences:  -7726822
columns differences:  -13


In [31]:
#Check the number of unique value of vessel's mmsi
unique_mmsi = df_123['shipId'].unique()
unique_mmsi.shape

(6778,)

#### 28. Create Records Id

In [48]:
#Add record_id as a primary key
df_123['rec_id'] = range(1, len(df_123) + 1)
df_123['rec_id']

0                1
1                2
2                3
3                4
4                5
            ...   
6653543    6631018
6653544    6631019
6653545    6631020
6653546    6631021
6653547    6631022
Name: rec_id, Length: 6631022, dtype: int64

#### 29. Rearrange the sequence of columns

In [51]:
df_123_new = df_123.reindex(columns=['rec_id', 'type', 'mmsi', 'status', 'turn', 'speed', 'accuracy', 'lon', 'lat', 'course', 'heading', 'time', 'date_', 'time_', 'shipId', 'Geo_Cell_ID'])

In [69]:
df_123_new

,rec_id,type,mmsi,status,turn,speed,accuracy,lon,lat,course,heading,time,date_,time_,shipId,Geo_Cell_ID
0,1,1,255805665,0,0,16.8,0,103.75723,1.13256,46.1,48,2019-12-01 00:00:30+00:00,2019-12-01,00:00,842,1718
1,2,1,255805665,0,0,16.1,0,103.78775,1.15364,59.4,60,2019-12-01 00:08:36+00:00,2019-12-01,00:08,842,1833
2,3,1,255805665,0,0,16.1,0,103.79072,1.15550,57.9,61,2019-12-01 00:09:01+00:00,2019-12-01,00:09,842,1834
3,4,1,255805665,0,0,16.0,0,103.83163,1.17622,70.0,69,2019-12-01 00:19:26+00:00,2019-12-01,00:19,842,1950
4,5,1,255805665,0,0,16.0,0,103.83324,1.17681,70.3,70,2019-12-01 00:19:49+00:00,2019-12-01,00:19,842,1950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6653543,6631018,1,477390400,0,0,2.1,0,103.77611,1.26823,9.7,17,2021-01-31 23:13:15+00:00,2021-01-31,23:13,3347,2448
6653544,6631019,1,477390400,0,0,1.9,0,103.77640,1.26950,12.1,16,2021-01-31 23:15:35+00:00,2021-01-31,23:15,3347,2448
6653545,6631020,1,477390400,0,0,1.6,0,103.77667,1.27108,7.3,13,2021-01-31 23:18:45+00:00,2021-01-31,23:18,3347,2504
6653546,6631021,1,477390400,0,0,0.9,0,103.77713,1.27450,359.4,8,2021-01-31 23:27:13+00:00,2021-01-31,23:27,3347,2504


#### 30. Rearrange the sequence of columns

In [108]:
df_123_new2[['day', 'month', 'year']] = df_123_new2[['day', 'month', 'year']].apply(lambda x: x.str.lstrip('0'))

C:\Users\Agus\AppData\Local\Temp\ipykernel_17936\1589071077.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_123_new2[['day', 'month', 'year']] = df_123_new2[['day', 'month', 'year']].apply(lambda x: x.str.lstrip('0'))


In [110]:
df_123_new2['new_date'] = df_123_new2.apply(lambda row: '/'.join(row[['day', 'month', 'year']]), axis=1)

C:\Users\Agus\AppData\Local\Temp\ipykernel_17936\2384475485.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_123_new2['new_date'] = df_123_new2.apply(lambda row: '/'.join(row[['day', 'month', 'year']]), axis=1)


In [114]:
df_123_new['new_date']

0          1/12/2019
1          1/12/2019
2          1/12/2019
3          1/12/2019
4          1/12/2019
             ...    
6653543    31/1/2021
6653544    31/1/2021
6653545    31/1/2021
6653546    31/1/2021
6653547    31/1/2021
Name: new_date, Length: 6631021, dtype: object

In [ ]:
df_123_new2[['day', 'month', 'year']] = df_123_new2[['day', 'month', 'year']].apply(lambda x: x.str.lstrip('0'))

# Display the modified DataFrame
print(df_123_new2)


In [ ]:
print('Final statistics Descritive of the cleaned data (type 1,2,3)\n')
df_123.describe().T

Final statistics Descritive of the cleaned data (type 1,2,3)



,count,mean,std,min,25%,50%,75%,max
type,6631022.0,1.383035e+00,7.861344e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.000000e+00
mmsi,6631022.0,5.096755e+08,1.063713e+08,2.054080e+08,4.775049e+08,5.630751e+08,5.656030e+08,7.102390e+08
speed,6631022.0,3.738638e+00,4.962701e+00,0.000000e+00,0.000000e+00,3.000000e-01,7.300000e+00,1.023000e+02
accuracy,6631022.0,4.676071e-01,4.989496e-01,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
lon,6631022.0,1.037119e+02,7.885579e-02,1.033800e+02,1.036830e+02,1.037187e+02,1.037644e+02,1.039400e+02
lat,6631022.0,1.223228e+00,5.087235e-02,8.301400e-01,1.192140e+00,1.225300e+00,1.261110e+00,1.387010e+00
course,6631022.0,1.903175e+02,1.083338e+02,0.000000e+00,9.130000e+01,2.008000e+02,2.852750e+02,3.600000e+02
heading,6631022.0,1.991079e+02,1.475523e+02,0.000000e+00,7.400000e+01,1.410000e+02,2.990000e+02,5.110000e+02
status,6631022.0,1.012448e+00,2.371854e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.500000e+01
turn,6631022.0,-1.343748e+01,5.152305e+01,-1.280000e+02,0.000000e+00,0.000000e+00,0.000000e+00,1.270000e+02


In [ ]:
#Number of unique values of each column in the dataframe
df_123.apply(lambda x: x.nunique())

type                 3
mmsi              6777
speed              735
accuracy             2
lon              55526
lat              32418
course            3601
heading            361
time           1830388
status              16
turn               227
shipId            6777
time_             1440
date_              124
Geo_Cell_ID       1608
dtype: int64

In [21]:
#Export to csv

#df_edit.to_json('output.json', orient='records', lines=True)
df_0711.to_csv('df_123_new7.csv', index=False)


# UNUSED SCRIPT

In [22]:
df_0711.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6631021 entries, 0 to 6631020
Data columns (total 16 columns):
 #   Column       Dtype  
---  ------       -----  
 0   rec_id       int64  
 1   type         int64  
 2   mmsi         int64  
 3   status       int64  
 4   turn         int64  
 5   speed        float64
 6   accuracy     int64  
 7   lon          float64
 8   lat          float64
 9   course       float64
 10  heading      int64  
 11  time         object 
 12  new_date     object 
 13  time_        object 
 14  shipId       int64  
 15  Geo_Cell_ID  int64  
dtypes: float64(4), int64(9), object(3)
memory usage: 809.5+ MB


In [19]:
df_0711 = df_0711.reindex(columns=['rec_id', 'type', 'mmsi', 'status', 'turn', 'speed', 'accuracy', 'lon', 'lat', 'course', 'heading', 'time', 'new_date', 'time_', 'shipId', 'Geo_Cell_ID'])

In [5]:
df_0711=df_0711.drop(columns=['date_','day','month','year'])

In [17]:
df_0711['new_date'].shape

(6631021,)

In [10]:
pd.DataFrame(df_0711['new_date'].unique()).sort_values()a

TypeError: DataFrame.sort_values() missing 1 required positional argument: 'by'

In [120]:
#Delete df for Jan and Dec to free memory if necessary
#del df_Dec19
#del df_Dec20
#del df_Jan21
#del df_Dec21

In [ ]:
#Change type data of df_123_new['Geo_Cell_ID] to integer  
#df_123_new['Geo_Cell_ID'] = df_123_new['Geo_Cell_ID'].astype(int)
#df_123_new[['accuracy','status', 'turn', 'heading', 'Geo_Cell_ID']] = df_123_new[['accuracy','status', 'turn', 'heading','Geo_Cell_ID']].astype(int)
 
            
 

C:\Users\Agus\AppData\Local\Temp\ipykernel_17936\876763419.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_123_new[['accuracy','status', 'turn', 'heading', 'Geo_Cell_ID']] = df_123_new[['accuracy','status', 'turn', 'heading','Geo_Cell_ID']].astype(int)


In [121]:
enc(1.389,103.939)

3136

In [122]:
# Drop rows when lon and lat outside the given range 
#df_123 = df_123[(df_123['lon'].between(103.38, 103.94)) & (df_123['lat'].between(0.83, 1.39))]

#def categorize_coordinates(lon, lat):
#    if not isinstance(lon, list):
#        lon = [lon]
#    if not isinstance(lat, list):
#        lat = [lat]
#
#    category = []
#    for i in range(len(lon)):
#        if 103.38 <= lon[i] < 103.94 and 0.83 <= lat[i] < 1.39:
#            category.append(1)  # Assign the integer label 1 for Category A
#        else:
#            category.append(2)  # Assign the integer label 2 for Category B
#    return category

# Example usage with single values
#lon = 103.381
#lat = 0.831
#result = categorize_coordinates(lon, lat)
#print(result)



In [123]:
# Drop rows when length of 'mmsi' other than 9 digits (Already performed when constructing vessel dimension) 
#df_123 = df_123[df_123['mmsi'].astype(str).map(len)==9]

# Comparison before and after
#a = df_123.shape

#diff_rows = a[0] - b[0]
#diff_cols = a[1] - b[1]
#print('df type 1,2,3')
#print('before: ',b)
#print('after : ', a)
#print('row differences: ', diff_rows)
#print('columns differences: ', diff_cols)
#print()


In [124]:
#Export 

#df_edit.to_json('output.json', orient='records', lines=True)
#df_123_new.to_csv('df_123_new3.csv', index=False)
#df_5.to_csv('df_5_r2.csv')

In [125]:
# Create DF with only type 5 inside  
df_5 = df[df['type'].isin([5])]

# Comparison before and after
#a5 = df_5.shape
##diff_rows = a5[0] - b[0]
#diff_cols = a5[1] - b[1]
#print('Data type 5')
#print('before: ',b)
#print('after : ', a5)
#print('row differences: ', diff_rows)
#print('columns differences: ', diff_cols)

In [126]:
# Drop rows with missing values in the specified columns
# df_edit = df.dropna(subset=['mmsi'])

# Display DataFrame
#b = df.shape
#a = df_edit.shape
#diff_rows = a[0] - b[0]
#diff_cols = a[1] - b[1]
#print('before: ',b)
#print('after : ', a)
#print('row differences: ', diff_rows)
#print('columns differences: ', diff_cols)

In [127]:
# Display the shape the DataFrames
#print('INITIAL DATA BEFORE CLEANING')
#print('df_Dec19.shape\t', df_Dec19.shape)
#print('df_Dec20.shape\t', df_Dec20.shape)
#print('df_Jan21.shape\t', df_Jan21.shape)
#print('df_Dec21.shape\t', df_Dec21.shape)
#print('\ndf.shape\t', df.shape)
#print("\nCount of unique df['mmsi']:", len(df['mmsi'].unique()))
#print("\nunique types:", sorted(df['type'].unique()))

#print(pd.DataFrame(df.info()))

In [128]:
#Using regular expression for correcting typos
#import re

# Example text data
#text = "Hello, my email is john.doe@example.com. Please contact me!"

# Remove email addresses from the text
#cleaned_text = re.sub(r'\S+@\S+', '', text)

# Display the cleaned text
#print(cleaned_text)


In [129]:
#Scrapping city names given a range of GPS coordinates
#pip install requests
#import requests

# Define the GPS coordinate range
min_lat, max_lat = 40.7128, 40.7740  # Replace with your minimum and maximum latitude range
min_lon, max_lon = -74.0060, -73.9787  # Replace with your minimum and maximum longitude range

# Define the OpenCage Geocoding API URL
#api_url = f'https://api.opencagedata.com/geocode/v1/json?q={min_lat}+{min_lon},{max_lat}+{max_lon}&key=YOUR-API-KEY'  # Replace 'YOUR-API-KEY' with your actual API key

# Make a request to the API
#response = requests.get(api_url)

#if response.status_code == 200:
#    data = response.json()
#    city_names = [result['components']['city'] for result in data['results'] if 'city' in result['components']]
#    print("City names in the GPS coordinate range:", city_names)
#else:
#    print(f"Failed to retrieve data. Status code: {response.status_code}")


In [130]:
# Drop rows with length of IMO > 7 digits  
#df_edit = df_edit[df_edit['imo'].astype(str).str.len().between(1, 7)]

# Comparison before and after
#b = df.shape
#a = df_123_edit.shape
#diff_rows = a[0] - b[0]
#diff_cols = a[1] - b[1]
#print('before: ',b)
#print('after : ', a)
#print('row differences: ', diff_rows)
#print('columns differences: ', diff_cols)